In [1]:
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from pykalman import KalmanFilter
def kfilter(array):
  if type(array)==str:
        array=eval(array)
  measurements = np.array(array)
  kf = KalmanFilter(transition_matrices=[1],
                  observation_matrices=[1],
                  initial_state_mean=measurements[0],
                  initial_state_covariance=1,
                  observation_covariance=80,
                  transition_covariance=1) 
  state_means, state_covariances = kf.filter(measurements) 
  state_means=list(state_means)
  for i in range(len(state_means)):
    state_means[i]=state_means[i][0]
  return state_means

In [2]:
import pandas as pd
import os
df = pd.read_csv('dataset_train.csv',sep=';')
def dfdel(value):
    return len(eval(value))
print(len(df))
df = df[df['Data'].map(dfdel)== 240]


31383


In [3]:
df.head()

,Filename,Test_index,Presentation,Question,Data,Data_2,Class_label
0,6ffe4933-3202-451b-ae6b-fee85cafc829,2,3,7,"[298, 296, 2, -455, -703, -596, -262, -44, 28,...","[-918, -1078, -450, 640, 1503, 1635, 1083, 589...",1
1,8de3189c-8e95-43a2-b6d2-743f097a57fb,0,1,0,"[99, 75, -228, -588, -583, -465, -394, -363, -...","[499, 494, 520, 577, 595, 584, 575, 567, 561, ...",0
2,c74f5292-c8e8-4ef2-a963-7d035e5e3330,3,3,1,"[-369, -543, -644, -670, -636, -586, -524, -44...","[263, 444, 548, 576, 539, 485, 419, 331, 266, ...",2
3,c856d82f-917e-461c-a91f-5298926b8b31,0,3,6,"[-6, -32, -152, -253, -275, -246, -211, -197, ...","[2, 138, 782, 1314, 1471, 1354, 1183, 1127, 10...",1
4,00b38188-82ee-4f1d-9661-485338815751,3,1,2,"[165, 178, 190, 202, 218, 230, 200, 121, 71, 8...","[-722, -711, -700, -686, -669, -665, -712, -79...",2


In [4]:
class0 = df[df['Class_label']==0]
class1 = df[df['Class_label']==1]
class2 = df[df['Class_label']==2]
data = pd.concat([class0[:6700],class1[:6700],class2[:6700]])

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['Data'], data['Class_label'], test_size=0.33, random_state=42)
y_test = np.array(y_test)
y_train = np.array(y_train)

In [6]:
X_train=list(X_train)
X_test=list(X_test)
def convX(X):
    for i in range(len(X)):
     X[i]=kfilter(list(eval(X[i])))
     if i%1000 ==0:
            print(str(i)+'/'+str(len(X)))
    return np.array(X)
print('Train')
x_train = convX(X_train)
print('Test')
x_test=convX(X_test)

Train
0/13467
1000/13467
2000/13467
3000/13467
4000/13467
5000/13467
6000/13467
7000/13467
8000/13467
9000/13467
10000/13467
11000/13467
12000/13467
13000/13467
Test
0/6633
1000/6633
2000/6633
3000/6633
4000/6633
5000/6633
6000/6633


In [7]:
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

In [8]:
num_classes = len(np.unique(y_train))
print(num_classes)

3


In [9]:
idx = np.random.permutation(len(x_train))
x_train = x_train[idx]
y_train = y_train[idx]

In [10]:
y_train[y_train == -1] = 0
y_test[y_test == -1] = 0

In [11]:
def make_model(input_shape):
    input_layer = keras.layers.Input(input_shape)

    conv1 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.ReLU()(conv1)

    conv2 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.ReLU()(conv2)

    conv3 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.ReLU()(conv3)

    gap = keras.layers.GlobalAveragePooling1D()(conv3)

    output_layer = keras.layers.Dense(num_classes, activation="softmax")(gap)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)


model = make_model(input_shape=x_train.shape[1:])
keras.utils.plot_model(model, show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [12]:
epochs = 500
batch_size = 32

callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model.h5", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001
    ),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1),
]
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)
history = model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_split=0.2,
    verbose=1,
)

Epoch 1/500
337/337 [==============================] - 15s 40ms/step - loss: 1.1114 - sparse_categorical_accuracy: 0.3462 - val_loss: 1.1082 - val_sparse_categorical_accuracy: 0.3315 - lr: 0.0010
Epoch 2/500
337/337 [==============================] - 13s 38ms/step - loss: 1.1009 - sparse_categorical_accuracy: 0.3559 - val_loss: 1.1093 - val_sparse_categorical_accuracy: 0.3296 - lr: 0.0010
Epoch 3/500
337/337 [==============================] - 13s 38ms/step - loss: 1.1014 - sparse_categorical_accuracy: 0.3446 - val_loss: 1.1050 - val_sparse_categorical_accuracy: 0.3463 - lr: 0.0010
Epoch 4/500
154/337 [============>.................] - ETA: 6s - loss: 1.1006 - sparse_categorical_accuracy: 0.3486

KeyboardInterrupt: 

In [ ]:
model = keras.models.load_model("best_model.h5")

test_loss, test_acc = model.evaluate(x_test, y_test)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

In [ ]:
metric = "sparse_categorical_accuracy"
plt.figure()
plt.plot(history.history[metric])
plt.plot(history.history["val_" + metric])
plt.title("model " + metric)
plt.ylabel(metric, fontsize="large")
plt.xlabel("epoch", fontsize="large")
plt.legend(["train", "val"], loc="best")
plt.show()
plt.close()

In [ ]:
y = model.predict(x_test)

In [ ]:
y

In [ ]:
y[0]

In [ ]:
def myprint(i):
 global y
 global y_test
 print(y[i])
 print(y_test[i])
 print('----')

In [ ]:
myprint(500)